In [ ]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

In [ ]:
!pip install tqdm
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [ ]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

In [ ]:
from keras.applications.xception import Xception
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

base_model = Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3))

# freeze the layers that we don't want to train
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D(input_shape=train_tensors.shape[1:])(x)
predictions = Dense(133, activation='softmax')(x)

merged_model = Model(inputs=[base_model.input], outputs=[predictions])

In [ ]:
merged_model.summary()

In [ ]:
merged_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

### (IMPLEMENTATION) Train the Model

Train your model in the code cell below.  Use model checkpointing to save the model that attains the best validation loss.  

You are welcome to [augment the training data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html), but this is not a requirement. 

In [ ]:
### TODO: Train the model.
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.Xception.hdf5', verbose=1, save_best_only=True)
merged_model.fit(train_tensors, train_targets, validation_data=(valid_tensors, valid_targets), epochs=20, batch_size=20, callbacks=[checkpointer], verbose=1)

### (IMPLEMENTATION) Load the Model with the Best Validation Loss

In [ ]:
### TODO: Load the model weights with the best validation loss.
merged_model.load_weights('saved_models/weights.best.Xception.hdf5')

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of dog images. Ensure that your test accuracy is greater than 60%.

In [ ]:
### TODO: Calculate classification accuracy on the test dataset.
Xception_predictions = [np.argmax(merged_model.predict(np.expand_dims(feature, axis=0))) for feature in test_tensors]

test_accuracy = 100 * np.sum(np.array(Xception_predictions) == np.argmax(test_targets, axis=1)) / len(Xception_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

### save the model for exporting to CoreML

In [ ]:
merged_model.save('saved_models/xception_full.hdf5')
with open('./saved_models/class_names.txt', 'w') as f:
    for name in dog_names:
        f.write(f"{name.replace('_', ' ').lower()}\n")

### convert the model to CoreML

In [ ]:
import coremltools

scale = 1 / 255.
coreml_model = coremltools.converters.keras.convert(
    './saved_models/xception_full.hdf5',
    input_names='image',
    image_input_names='image',
    output_names='output',
    class_labels='./saved_models/class_labels.txt',
    image_scale=scale)

print(coreml_model)

coreml_model.author = 'Jeremy Jones'
coreml_model.license = 'MIT'
coreml_model.short_description = 'Model to classify dog breeds.'

coreml_model.input_description['image'] = 'RGB image'
coreml_model.output_description['output'] = 'Predicted dog breed'

coreml_model.save('DogBreedClassifier.mlmodel')